### GEE Batch Downloading Script
*Developed By Puzhao Zhang

*Modified By Xikun Hu

In [2]:
import ee, os, zipfile
import numpy as np
# from url_file_download import url_download
import matplotlib.pyplot as plt
from skimage import data, img_as_float
from skimage.segmentation import chan_vese
import os
import urllib.request as request
import logging
import sys
import ee.mapclient

In [3]:
from IPython.display import Image

# Initialize the Earth Engine module.
ee.Initialize()

# Display a thumbnail of a sample image asset.
Image(url=ee.Image('CGIAR/SRTM90_V4').getThumbUrl({'min': 0, 'max': 3000}))

In [4]:
def url_download(url, savePath, saveName):

    logging.basicConfig(
        format='%(asctime)s %(levelname)s %(message)s',
        level=logging.INFO,
        stream=sys.stdout)
    
    # file_path = os.path.join(os.getcwd(),'dir_name/file_name')
    filePath = savePath + saveName

    if os.path.isfile(filePath):
        os.system("rm " + filePath)
        logging.info("Existed file deleted: " + saveName)
    else:
        logging.info("File doesn't exist.")
    # replace with url you need

    # if dir 'dir_name/' doesn't exist
    file_dir = savePath
    if not os.path.exists(file_dir):
        logging.info("Make direction: " + savePath)
        os.mkdir(file_dir)

    def down(_save_path, _url):
        try:
            request.urlretrieve(_url, _save_path)
            return True
        except:
            print('\nError when retrieving the URL:\n{}'.format(_url))
            return False

    # logging.info("Downloading file.")
    #
    # flag = False
    # while(~flag):
    #     flag = down(filePath, url)

    flag = down(filePath, url)
    if flag:
        print("------- Download Finished! ---------")
    else:
        print("------- Download Fail --------")



In [7]:
def set_timeEnd_newdays(img):
    group_days = img.date().format().slice(0, 10)
    return img.set('system:time_end', group_days)

# "group by" date
def group_days(imgcollection):
    imgCol_sort = imgcollection.sort("system:time_start")
    imgCol = imgCol_sort.map(set_timeEnd_newdays)
    d = imgCol.distinct(['system:time_end'])
    di = ee.ImageCollection(d)
    date_eq_filter = (ee.Filter.equals(leftField= 'system:time_end',
                                       rightField ='system:time_end'))

    saveall = ee.Join.saveAll("to_mosaic")
    j = saveall.apply(di, imgCol, date_eq_filter)
    ji = ee.ImageCollection(j)
    original_proj = ee.Image(ji.first()).select(0).projection()


    def mosaicImageBydate(img):
        mosaiced = ee.ImageCollection.fromImages(img.get('to_mosaic')).mosaic().copyProperties(img, img.propertyNames())
        return ee.Image(mosaiced)

    imgcollection_grouped = ji.map(mosaicImageBydate)

    return ee.ImageCollection(imgcollection_grouped.copyProperties(imgCol, imgCol.propertyNames()))

In [8]:
def un_zip(dataPath, dataName, unzipPath):
    """unzip zip file"""
    zip_file = zipfile.ZipFile(dataPath + dataName + ".zip")
    if os.path.isdir(unzipPath):
        pass
    else:
        os.mkdir(unzipPath)
    for names in zip_file.namelist():
        zip_file.extract(names, unzipPath)
    zip_file.close()

In [9]:
def batchReName(dataPath, savePath, saveName):
    print("dataPath: " + dataPath)
    for file in os.listdir(dataPath):
        format = "." + file.split(".")[-1]
        orginalName = file.split(".")[0]
        saveFileName = file.replace(orginalName, saveName)

        if format == ".tif" and ('angle' not in file):
            if os.path.isfile(savePath + saveFileName):
                os.system("rm " + savePath + saveFileName)
            os.rename(dataPath + file, savePath + saveFileName)

In [12]:
def batch_data_download_GEE(downLoadList, pathKeyWord, scale, bands4download, toRGBflag):

    savePath = rootPath + "{:}_{:}m_{:}_GEE_download_zip/".format(fireName, scale, pathKeyWord)
    tifPath = rootPath + "{:}_{:}m_{:}_Tif_collection/".format(fireName, scale, pathKeyWord)
    pngPath = rootPath + "{:}_{:}m_{:}_PNG_collection/".format(fireName, scale, pathKeyWord)
    rgbPath = rootPath + "{:}_{:}m_{:}_RGB_collection/".format(fireName, scale, pathKeyWord)
    
    print("downloadList: {}".format(downLoadList))

    for saveName in downLoadList:

        img = globals()[saveName]
        img = ee.Image(img)

        if len(bands4download) > 0:
            url = (img.select(bands4download)
                .getDownloadUrl({
                # 'image': img.select(['B11', 'B12']),
                # 'description': "creek_wildFire_S2_20171130",
                'region': cor,
                'scale': scale,
                'crs': "EPSG:32610"
#                 'crs': "EPSG:32610"
#                 EPSG:3857,4326
            }))
        else:
            url = (img
                .getDownloadUrl({
                # 'image': img.select(['B11', 'B12']),
                # 'description': "creek_wildFire_S2_20171130",
                'region': cor,
                'scale': scale,
#                 'crs': "EPSG:3857",                
                'crs': "EPSG:32610"
                # EPSG:3857,4326
            }))

        if not os.path.exists(savePath):
            os.makedirs(savePath)

        print("{}:\n {}".format(saveName, url))
#         print("savePath:", savePath)
        url_download(url=url, savePath=savePath, saveName=saveName + ".zip")

        # run = ee.batch.Export.image()

        unzipPath = savePath + "unzipedFiles/"
        un_zip(savePath, saveName, unzipPath)

        renamePath = savePath + "renamedFiles/"
        renamedName = saveName

        os.system("rd/s/q " + renamePath)

        if not os.path.exists(renamePath):
            os.makedirs(renamePath)
        batchReName(unzipPath, renamePath, renamedName)

        # ======================================================

        # os.system("rd/s/q " + tifPath)  # remove directory
        # shutil.copytree(renamePath, tifPath)  # move directory

        # from snappy_tif_processing import tif2png, tif2snapTif
        from gdal_tif2rgb import tifBand2png_GDAL, bandsMerge2tif

        bandsMerge2tif(renamePath, saveName, tifPath, saveName, stretchFlag=True)

        for file in os.listdir(renamePath):
            fileName = file[:-4]
            tifBand2png_GDAL(renamePath, fileName, pngPath, fileName)
            # shutil.copy(renamePath + fileName + ".tif", tifPath)



            # tif2snapTif(renamePath, fileName, tifPath) # copy tif files

        # ## ============== delete directory =====================
        os.system("rd/s/q " + unzipPath)
        # os.system("rd/s/q " + renamePath)
        # os.system("rm " + savePath + saveName + ".zip")


        if toRGBflag:
            from bands2rgb import pngBand2rgb
            if satName == 'S2':
                rgbBands = ['B8', 'B2', 'B3']
            elif satName == 'L8':
                rgbBands = ['B7', 'B6', 'B7']
            pngBand2rgb(pngPath, saveName, rgbPath, saveName, bands=rgbBands)

In [14]:
if __name__ == "__main__":

    ee.Initialize()

    # creekFire = ee.Geometry.Rectangle([-118.4766, 34.197, -118.2582, 34.365])
    # creekFire = ee.Geometry.Rectangle([-118.4766, 34.197, -118.2582, 34.365])
    creekFire = ee.Geometry.Rectangle([-118.4766, 34.197, -118.2335, 34.3927])

    venturaFireCenter = ee.Geometry.Rectangle([-119.3802, 34.323, -119.1385, 34.5088])

    thomasFireRightRoi = ee.Geometry.Rectangle([-119.063, 34.3548, -118.8714, 34.5145])
    thomasFireRightCord = [[-119.063, 34.3548], [-119.063, 34.5145], [-118.8714, 34.5145], [-118.8714, 34.3548]]

    thomasCord0 = [[-119.7345, 34.2277], [-119.7345, 34.6524], [-118.8913, 34.6524], [-118.8913, 34.2277]]

    ranchFire = ee.Geometry.Rectangle([-123.3456, 38.864, -122.2937, 39.6487])
    ranchFireCor = [[-123.3456, 38.864], [-123.3456, 39.6487], [-122.2937, 39.6487], [-122.2937, 38.864]]

    carrFire_v0 = ee.Geometry.Rectangle([-122.9928, 40.4176, -122.2347, 40.9949])
    carrFireCor_v0 = [[-122.9928, 40.4176], [-122.9928, 40.9949], [-122.2347, 40.9949], [-122.2347, 40.4176]]

    carrFire = ee.Geometry.Rectangle([-122.8534, 40.4424, -122.3288, 40.9695])
    carrFireCor = [[-122.8534, 40.4424], [-122.8534, 40.9695], [-122.3288, 40.9695], [-122.3288, 40.4424]]

    thomasFire = ee.Geometry.Rectangle([-119.7345, 34.2277, -118.8913, 34.6524])
    thomasCor = [[-119.7345, 34.2300], [-119.7345, 34.6524], [-118.8913, 34.6524], [-118.8913, 34.2300]]

    elephantFire = ee.Geometry.Rectangle([-121.7697, 50.6512, -120.7068, 51.5224])
    elephantCor = [[-121.7697, 50.6512], [-120.7068, 50.6512], [-121.7697, 51.5224], [-120.7068, 51.5224]]
    
    gislavedFire = ee.Geometry.Rectangle([13.50, 57.25, 13.60, 57.29])
    gislavedCor = [[13.50, 57.25], [13.50, 57.29], [13.60, 57.29], [13.60, 57.25]]
    
    # There are something errors if the corrodinates have too many decimal points
    
    roi = elephantFire
    cor = elephantCor
    
#     ee.mapclient.centerMap(-119.31338418793973,34.53954088137944,14)
    

    ### ==================== Optical Images ========================
    t1 = '2017-10-02'
    t2 = '2017-10-04'

    # S1 = (ee.ImageCollection("COPERNICUS/S1_GRD")
    #     .filterDate(t1, t2)
    #     .filterBounds(roi)
    #     .sort('system:time_start'))

    S2 = (ee.ImageCollection("COPERNICUS/S2")
          .filterDate(t1, t2)
          .filterBounds(roi)
          .sort('system:time_start'))

    # -----------------------------------
    satName = 'S2'
    col = globals()[satName]#.merge(L8)
    printImgIdBeforeMosaicFlag = True

    fireName = 'Elephant_834_'
    preFix = 'MSI'
    scale = 20
    dNBR_Flag = False
    rootPath = "/home/jovyan/work/workspace/Xikun_Reps/GEE_app/GEE_dl/"

    # ===================================

    if printImgIdBeforeMosaicFlag:
        print("-------------------------------------------------------")
        num0 = col.size().getInfo()
        print("Size of ImageCollection before Mosaicing: {}".format(num0))
        print("------------------- Image id List  --------------------")
        col_list = col.toList(num0)
        for idx in range(num0):
            img = ee.Image(col_list.get(idx))
            if 'S2' in satName:
                date = img.get("system:index").getInfo()[:8]
            elif 'L8' in satName:
                date = img.get("system:index").getInfo()[12:]
            else:
                date = None
                print("------------- No Matched Satellite ------------")
            # print(date)

            varName = "{}_{}_{}".format(preFix, date, satName)
            
    print("--------------end--------------")

    # ===================================

    col_groupedBydate = group_days(col)
    # print("col_groupedBydate", col_groupedBydate)
    num = col_groupedBydate.size().getInfo()
    
    print("-------------------------------------------------------")
    print("Size of ImageCollection after Mosaicing: {}".format(num))
    print("-------------------- Download List --------------------")
    # num = 10
    dataList = col_groupedBydate.toList(num)
    # print(dataList)

    msiDownLoadList = []

    for i in range(0, num):
        img = ee.Image(dataList.get(i))

        # if img.get("SPACECRAFT_ID").getInfo() == 'LANDSAT_8':
        #     satName = 'L8'
        # elif img.get("PRODUCT_ID").getInfo()[:2] == 'S2':
        #     satName = 'S2'
        # print("Satellite: {}".format(satName))

        if 'S2' in satName:
            date = img.get("system:index").getInfo()[:8]
        elif 'L8' in satName:
            date = img.get("system:index").getInfo()[12:]
        else:
            date = None
            print("------------- No Matched Satellite ------------")
        # print(date)

        varName = "{}_{}_{}".format(preFix, date, satName)
            
#         else:
        globals()[varName] = img
        msiDownLoadList.append(varName)
        print("{}".format(varName))

    num = len(msiDownLoadList)
    print("Number of optical images: {}".format(num))
    print("----------------------------------------------------------")

    if satName == 'S2':
        swirBands = ['B2','B3','B8']
    elif satName == 'L8':
        swirBands = ['B6', 'B7']
    else:
        swirBands = []


#     print("downloadBands: {}".format(downloadBands))
    batch_data_download_GEE(downLoadList=msiDownLoadList, pathKeyWord="MSI", scale=scale,
                            bands4download = swirBands, toRGBflag=True)

-------------------------------------------------------
Size of ImageCollection before Mosaicing: 7
------------------- Image id List  --------------------
--------------end--------------
-------------------------------------------------------
Size of ImageCollection after Mosaicing: 1
-------------------- Download List --------------------
MSI_20171003_S2
Number of optical images: 1
----------------------------------------------------------
downloadList: ['MSI_20171003_S2']
MSI_20171003_S2:
 https://earthengine.googleapis.com/api/download?docid=0c87b080fc7b2ea39e707325f8d68642&token=ec2400d0567e17f7ba707bea8b222024
2019-08-21 13:57:06,858 INFO Existed file deleted: MSI_20171003_S2.zip
------- Download Finished! ---------
dataPath: /home/jovyan/work/workspace/Xikun_Reps/GEE_app/GEE_dl/Elephant_834__20m_MSI_GEE_download_zip/unzipedFiles/
['B8', 'B2', 'B3']
MSI_20171003_S2.B8
MSI_20171003_S2.B2
MSI_20171003_S2.B3
